<a href="https://colab.research.google.com/github/jgbrenner/psychometrics/blob/main/psychometricsC9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install numpy pandas matplotlib
!pip install rpy2
!pip install transformers
!pip install groq
!pip install groqflow

In [2]:
%load_ext rpy2.ipython


In [3]:
import openai

from google.colab import userdata


openai_api_key = userdata.get('OPENAI_API_KEY')


In [4]:
from groq import Groq

groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq client
client = Groq(api_key=groq_api_key)

# Define the messages for the LLM
messages = [
    {"role": "system", "content": "You are an expert psychometrician creating test items."},
    {"role": "user", "content": "Generate two items for the construct of Rigid Perfectionism: one regular-keyed and one reverse-keyed. Make them concise and clear."}
]

# Create the completion request
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Process the response
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Here are two test items measuring the construct of Rigid Perfectionism:

Regular-keyed item:

"I often feel frustrated when others don't meet my high standards."

(Scale: 1 - Strongly disagree to 5 - Strongly agree)

This item is regular-keyed because it is reflecting the common pattern of rigid perfectionists, who tend to experience negative emotions when others don't meet their high standards.

Reverse-keyed item:

"In situations where I can't achieve perfection, I'm able to adapt and move on without too much fuss."

(Scale: 1 - Strongly disagree to 5 - Strongly agree)

This item is reverse-keyed because it is soliciting information about rigid perfectionists' tendency to be flexible and move on when they can't meet their high standards, which is the opposite of their typical behavior.

Note: In both cases, the scale should ideally be anchored with Strongly disagree, Disagree, Neutral, Agree, and Strongly agree to allow respondents to accurately rate their level of agreement or disag

In [5]:
items = [
    {
        "item": "I often feel anxious when I make a mistake, even if it's a minor one, and I worry that it will affect my overall performance.",
        "type": "regular-keyed"
    },
    {
        "item": "I am comfortable with the idea that I may not be perfect all the time, and I don't get too upset when I make a mistake or things don't go exactly as planned.",
        "type": "reverse-keyed"
    }
]

import pandas as pd
items_df = pd.DataFrame(items)
items_df.to_csv("rigid_perfectionism_items.csv", index=False)


In [6]:
import requests
import numpy as np

embedding_endpoint = "https://api.openai.com/v1/embeddings"
embedding_model = "text-embedding-3-small"

headers = {
    "Authorization": f"Bearer {openai_api_key}",
    "Content-Type": "application/json"
}

embeddings = []
for item in items_df["item"]:
    response = requests.post(
        embedding_endpoint,
        headers=headers,
        json={"model": embedding_model, "input": item}
    )

    # Debugging: Print the full response
    print(response.json())

    if 'data' in response.json():
        embeddings.append(response.json()["data"])
    else:
        print(f"Error: {response.json().get('error', {}).get('message', 'Unknown error')}")

# Convert embeddings into a usable format
embeddings_array = np.array(embeddings)

{'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.024861827, -0.021199256, 0.02111436, 0.039924454, -0.0067733317, 0.017221363, 0.039439347, 0.019028394, -0.05782497, 0.0584071, -0.008392382, -0.008519724, -0.0011043806, 0.0016357263, 0.0012529452, 0.03856615, 0.0013962038, -0.004866248, 0.02532268, 0.009799198, 0.034175918, 0.020156272, 0.024025014, 0.009301961, 0.042374257, -0.029397594, 0.01702732, -0.01622689, 0.049068756, 0.034515493, 0.02665673, -0.019343713, -0.029712915, -0.04222872, 0.015329438, 0.017936898, 0.029300572, -0.0066035436, 0.015438588, -0.017609451, -0.037110824, 0.040142756, 0.032211225, -0.02505587, 0.037013803, -0.040045734, -0.017075831, 0.008234723, -0.0121762315, 0.02862142, -0.052343242, 0.06306414, 0.007883018, 0.091734074, 0.030367812, -0.007373654, 0.0031956544, -0.0067733317, 0.013255598, 0.0155841205, 0.0329874, 0.011921549, 0.002904589, 0.012903894, -0.019343713, -0.0102843065, 0.030319301, -0.025468213, 0.012061018, 0.00

In [ ]:
%%R

# Set the custom library path to persist during the Colab session
custom_lib <- "/content/R_custom_lib"
dir.create(custom_lib, showWarnings = FALSE)
.libPaths(custom_lib)

# Install devtools
if (!"devtools" %in% rownames(installed.packages())) {
    install.packages("devtools", lib = custom_lib)
}

# Use devtools to install EGAnet from GitHub
library(devtools)
install_github("hfgolino/EGA", lib = custom_lib)

# Load EGAnet library
library(EGAnet)





These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

1: All                                
2: CRAN packages only                 
3: None                               
4: fontawesome (0.5.2 -> 0.5.3) [CRAN]

Enter one or more numbers, or an empty line to skip updates: 1
coda         (NA    -> 0.19-4.1  ) [CRAN]
statnet.c... (NA    -> 4.10.0    ) [CRAN]
mvtnorm      (NA    -> 1.3-2     ) [CRAN]
rpf          (NA    -> 1.0.14    ) [CRAN]
BH           (NA    -> 1.84.0-0  ) [CRAN]
StanHeaders  (NA    -> 2.32.10   ) [CRAN]
RcppEigen    (NA    -> 0.3.4.0.2 ) [CRAN]
RcppParallel (NA    -> 5.1.9     ) [CRAN]
nloptr       (NA    -> 2.1.1     ) [CRAN]
minqa        (NA    -> 1.2.8     ) [CRAN]
RUnit        (NA    -> 0.4.33    ) [CRAN]
openxlsx     (NA    -> 4.2.7.1   ) [CRAN]
plyr         (NA    -> 1.8.9     ) [CRAN]
kutils       (NA    -> 1.73      ) [CRAN]
carData      (NA    -> 3.0-5     ) [CRAN]
lme4         (NA    -> 1

Streaming output truncated to the last 5000 lines.




































































































































































































































































































































































































































In [8]:
%%R
# Load the EGAnet library (formerly EGA)
library(EGAnet)



Error in library(EGAnet) : there is no package called ‘EGAnet’


In [9]:
%R -i embeddings_array

%%R
library(EGAnet)

# Perform Exploratory Graph Analysis (EGA)
ega_result <- EGA(as.matrix(embeddings_array))
print(ega_result)


SyntaxError: invalid syntax (<ipython-input-9-5a8d46495c34>, line 7)